In [1]:
# imports
from collections import Counter
# from itertools import islice
import re
import pandas as pd
import string
import csv
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import math
import numpy as np
porter = PorterStemmer()
# #########################
# DATA CLEANING TECHNIQUES
# #########################
# split by sentance
# remove puncuation
# lowercase letters
# remove/convert numbers
# -------------------------

# #########################
# NLP TECHNIQUES
# #########################
# stemming
# -------------------------

In [15]:
stopwords = {"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "between", "into", "through", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "that", "thats", "that's", "than", "too", "very", "can", "will", "just", "should", "now"}
# sent1 = "i am sam"
# sent2 = "sam i am"
# sent3 = "i do not like green eggs and ham"
# sent4 = "the quick person did not realize his speed and the quick person bumped "


# # words = re.findall("\w+", sent4)
# # bigramCount = dict(Counter(zip(words, islice(words, 1, None))))

# # for i in bigramCount.keys():
# #     print(str(i) + " count: %d" % bigramCount[i])

# load in the crawled data and perform some cleaning
news = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\newsutf8.csv")
print(news.shape[0])
# news = news[:100]
for i in range(news.shape[0]):
    s = news.iloc[i][3][2:]
    s2 = news.iloc[i][1][2:]
    news.at[i,'body'] = s
    news.at[i,'headline'] = s2

print("done")

# print(news.iloc[0][1])
# print(news.iloc[1][1])
# print(news.iloc[2][1])
# print(news.iloc[3][1])

7613
done


In [16]:
gibberish = ['\(CNN\)', 'CNN', 'Getty Images', '\\\\xc2', '\\\\xb0', '\\\\xc3', '\\\\x94', '\\\\x9c', '\\\\x9d', '\\\\x95', '\\\\xb3', '\\\\xe2', '\\\\x80', '\\\\x99', '\\\\xa9', '\\\\xf0', '\\\\x9f', '\\\\x91']
for i in range(len(gibberish)):
    for j in range(news.shape[0]):
        output = news.iloc[j][3]
        output = re.sub(gibberish[i], '', output)
        output = re.sub('\\\\n', ' ', output)
        output = re.sub('\\\\\\\'', '', output)
        output = re.sub('\'', '', output)
        output = re.sub('Read More', '', output)
        news.at[j,'body'] = output

print("#################################################")



#################################################


In [17]:
collection = str.maketrans('', '', string.punctuation)
dterms = dict()
for j in range(news.shape[0]):    
    tokens = word_tokenize(news.iloc[j][3])
    tokens = [t.lower() for t in tokens]
    depunc = [t.translate(collection) for t in tokens]
    words = [w for w in depunc if not w  == '']
    cleaner = [w for w in words if not w in stopwords]
    stemmed = [porter.stem(word) for word in cleaner]
    dterms[str(j)+news.iloc[j][1]] = stemmed
#     dterms[j] = cleaner


In [18]:
term_list = []
doc_index = {}
term_index = {}
words_per_doc = {}


#function that creates termlist and doc_idx as we count each doc_length
def populate_data_structs():
#     for doc in range(len(dterms)):
    for doc in dterms.keys():
        count = 0
        words_per_doc[doc] = set({})
        for word in dterms[doc]:
            count += 1
            term_list.append((word, doc))
            words_per_doc[doc].add(word)
        doc_index[doc] = count

def update_term(item):
    i = 0
    for listitem in term_index[item[0]]:
        #if word_docName == encounterd_fileID
        if (item[1] == listitem[0]):
            listitem[1] += 1
            return
        else:
            i += 1
    term_index[item[0]].append( [item[1] , 1 ] )
    return


def make_term_index():
    for item in term_list:
        if item[0] not in term_index:
            # add term
            term_index[item[0]] = [[item[1], 1]]
        else:
            # update term
            update_term(item)
            
populate_data_structs()
make_term_index()

In [19]:
def termweight(term, docno):
    if term in words_per_doc[docno]:
        for postings in term_index[term]:
            if postings[0] == docno:
#                 tf = postings[1]/doc_index[postings[0]] #num_occur/doc_size
#                 idf = math.log(len(doc_index)/len(term_index[term]), 2)
#                 tf_idf = tf*idf
#                 print('Posting: Doc# ' + str(postings[0]) + ', tf: ' + str(tf) + ', idf: ' + str(idf) + ', tf-idf: ' + str(tf_idf))
                return postings[1]/doc_index[postings[0]] * math.log(len(doc_index)/len(term_index[term]), 2)
#     print('Posting: Doc# ' + str(docno) + ', tf: ' + str(0))
    return 0


term_doc_matrix = {}

for term in term_index.keys():
    row = []
    for doc in doc_index.keys():
        row.append(termweight(term, doc))
    term_doc_matrix[term] = row


keys = list(doc_index.keys())
# print(keys)

df = pd.DataFrame.from_dict(term_doc_matrix, orient='index', columns=keys)
df.to_csv (r'C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFMatrix.csv', index = True, header=True)

term_list = []
doc_index = {}
term_index = {}
words_per_doc = {}
term_doc_matrix = {}


In [20]:


def get_posting_list(term):
    return term_index[term] if term in term_index else []

def get_q_weight(term, query_words):
    tf = 0
    for w in query_words:
        if w.lower() == term:
            tf += 1
    tf = tf / len(query_words)
    t = ps.stem(term)
    if t in term_index:
        idf = 1 + math.log(len(doc_index)/len(term_index[t]))
    else:
        idf = 1
    return tf * idf



def get_d_weight(term, pair):
    if term in term_index:
        #postings = self.term_index[term]
        tf = pair[1]/doc_index[pair[0]] #num_occur/doc_size
        idf = 1 + math.log(len(doc_index)/len(term_index[term]))
        tf_idf = tf * idf
    else:
        tf_idf = 0
    return tf_idf


def cosine_score(query_words, inv_idx):
    scores = dict()
    for doc in inv_idx.doc_index.keys():
        scores[doc] = 0
    
    for t in query_words:
        t = t.lower()
        term = ps.stem(t)
        postings = inv_idx.get_posting_list(term)
        q_t_weight = inv_idx.get_q_weight(t, query_words)


        for pair in postings:
            d_t_weight = inv_idx.get_d_weight(term, pair)
            scores[pair[0]] += q_t_weight * d_t_weight
        
    for doc in inv_idx.doc_index.keys():
        scores[doc] = scores[doc] / inv_idx.doc_index[doc]
        
    sorted_scores = sorted(scores.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_scores

In [7]:
# for k in dterms.keys():
#     print(k, dterms[k][0])

a_file = open("TermDocMatrix.csv", "w")

writer = csv.writer(a_file)
for key, value in term_index.items():
    writer.writerow([key, value])

a_file.close()


In [2]:
# initialize the set and the dataframe with the first word of the first document
# words_encountered = set(dterms[0][0])
# unigrams = pd.DataFrame({dterms[0][0]: [0]})

# for i in range(news.shape[0]):
#     if i > 0:
#         tdf = pd.DataFrame([[0]*len(unigrams.columns)], columns=unigrams.columns)
#         t = unigrams.append(tdf,ignore_index=True)
#         unigrams = t
#     for word in dterms[i]:
#         if word not in words_encountered:
#             words_encountered.add(word)
#             col_index = len(unigrams.columns)
#             temp = pd.DataFrame({word: [0]*(i+1)})
#             t2 = unigrams.join(temp)
#             unigrams = t2
#             unigrams.at[i, word] = 1
#         else:
#             cols = list(unigrams.columns)
#             unigrams.at[i, word] += 1
# too slow with dataframes ^ 

# for i in range(10):
#     print(dterms[i][:6])

# unigrams.to_csv('unigrams2.csv', encoding='utf-8')




In [3]:
weightmatrix = pd.read_csv(r"C:\Users\Rogith\Desktop\Notebooks\newsgrams\TFIDFMatrix.csv")
print(weightmatrix.shape)

(53740, 7614)


In [5]:
display(weightmatrix.head())

,Unnamed: 0,0This Alaskan man is making a 14-hour boat trip to Costco every week to supply his small city with groceries amid the pandemic',1State Department inspector general becomes the latest watchdog fired by Trump',"2Inside New York's notorious Rikers Island jails, 'the epicenter of the epicenter' of the coronavirus pandemic""","3Warren Buffett's Berkshire Hathaway sells majority of stake in Goldman Sachs""","4Bundesliga marks soccer's elite return, with changes, during coronavirus pandemic""",5Mass gatherings are making their way back into US life but may not look the way we remember',6CNN to air two-hour special honoring the class of 2020',"7Protectors of the French language says 'Covid' takes a feminine article""",8Tensions rise between the White House and CDC as Birx critiques virus tracking',...,"7603Newly discovered virus takes more lives, spreads'","7604New SARS-like virus is a 'threat to the entire world'""",7605Health care workers sickened by new virus',7606Saudi Arabia reports 4 new coronavirus cases',7607Second case of challenging virus reported in France',7608Dangerous virus newly found in humans diagnosed in France',7609Death toll from new SARS-like virus climbs to 9',7610Hong Kong and SARS: A city under siege',7611Experts: New SARS-like virus could show up in U.S.',7612Boy in Hong Kong hospital tests negative for SARS-like illness'
0,one,0.008623,0.00081,0.0,0.0,0.005121,0.002431,0.0,0.0,0.008638,...,0.0,0.002352,0.007136,0.0,0.00567,0.01177,0.003089,0.003298,0.002486,0.0
1,helluva,0.134315,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.0
2,costco,0.233375,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.0
3,run,0.052258,0.00000,0.0,0.0,0.000000,0.003683,0.0,0.0,0.004362,...,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.0
4,grocer,0.082232,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.0
